In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_path = os.path.join(os.path.dirname('__file__'), 'aruba', 'data.txt')

database = []
with open(data_path, 'r') as file:
  for line in file.readlines():
    data = line.strip().split()
    
    database.append(line.strip().split())


df = pd.DataFrame(database)
df.columns = ['date', 'time', 'sensor', 'reading', 'activity', 'note']
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
df.sensor = df.sensor.str.strip()

df

,date,time,sensor,reading,activity,note
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin
1,2010-11-04,00:03:57.399391,M003,OFF,None,None
2,2010-11-04,00:15:08.984841,T002,21.5,None,None
3,2010-11-04,00:30:19.185547,T003,21,None,None
4,2010-11-04,00:30:19.385336,T004,21,None,None
...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.28507,T002,25.5,None,None
1719554,2011-06-11,23:48:02.888409,T001,23.5,None,None
1719555,2011-06-11,23:48:02.988798,T002,25,None,None
1719556,2011-06-11,23:53:06.4292,T002,25.5,None,None


## EDA

In [3]:
all_activities = sorted(x for x in df.activity.unique() if type(x) == str)

all_activities

['Bed_to_Toilet',
 'Eating',
 'Enter_Home',
 'Housekeeping',
 'Leave_Home',
 'Meal_Preparation',
 'Relax',
 'Respirate',
 'Sleeping',
 'Wash_Dishes',
 'Work']

In [4]:
df.activity.value_counts() /2

activity
Relax               2919.0
Meal_Preparation    1606.0
Leave_Home           431.0
Enter_Home           431.0
Sleeping             401.0
Eating               257.0
Work                 171.0
Bed_to_Toilet        157.0
Wash_Dishes           65.0
Housekeeping          33.0
Respirate              6.0
Name: count, dtype: float64

--- Note: The values of Relax doesn't match the READ ME.

In [5]:
display(df[df.duplicated(subset=['date', 'time', 'sensor', 'activity'], keep=False)].sort_values(by=['sensor', 'date', 'time', 'activity']))

duplicates = df[df.duplicated(subset=['date', 'time', 'sensor', 'activity'])].sort_values(by=['sensor', 'date', 'time', 'activity'])

# check if the activities are duplicated
print("Duplicated activities frequency table:")
display(duplicates.activity.value_counts())

,date,time,sensor,reading,activity,note
1566109,2011-05-23,07:33:38.313549,D001,OPEN,None,None
1572534,2011-05-23,07:33:38.313549,D001,OPEN,None,None
1566111,2011-05-23,07:33:47.501744,D001,CLOSE,None,None
1572536,2011-05-23,07:33:47.501744,D001,CLOSE,None,None
1566998,2011-05-23,08:45:34.23655,D004,OPEN,Leave_Home,begin
...,...,...,...,...,...,...
1576595,2011-05-23,18:10:59.326994,T005,22,None,None
1571217,2011-05-23,21:38:29.747715,T005,21.5,None,None
1577642,2011-05-23,21:38:29.747715,T005,21.5,None,None
1571283,2011-05-23,22:59:28.011756,T005,21,None,None


Duplicated activities frequency table:


activity
Relax               24
Meal_Preparation    20
Leave_Home           8
Enter_Home           8
Sleeping             6
Eating               4
Bed_to_Toilet        2
Wash_Dishes          2
Name: count, dtype: int64

In [6]:
# check for completely empty rows
print("Completely empty rows:")
display(df[df.isna().sum(axis=1) == 6])

# check for empty values for specific columns
print("Empty values in 'date', 'time', 'sensor', and 'reading' columns:")
display(df[['date', 'time', 'sensor', 'reading']].isna().sum())

Completely empty rows:


,date,time,sensor,reading,activity,note


Empty values in 'date', 'time', 'sensor', and 'reading' columns:


date       0
time       0
sensor     0
reading    0
dtype: int64

In [7]:
print("Activities followed by empty notes:")
display(df[df.activity.notna() & df.note.isna()])

Activities followed by empty notes:


,date,time,sensor,reading,activity,note


In [8]:
print("Create frequency table for time format values")
display(df.time.str.len().value_counts())

print("Check for time format less than specific characters")
display(df[df.time.str.len() < 11])

Create frequency table for time format values


time
15    1547978
14     154260
13      15635
12       1553
11        124
10          6
8           2
Name: count, dtype: int64

Check for time format less than specific characters


,date,time,sensor,reading,activity,note
558679,2011-01-15,13:21:29.6,M007,ON,None,None
679536,2011-01-29,09:22:15,M013,ON,None,None
1134647,2011-03-27,11:56:12.9,M022,OFF,None,None
1229596,2011-04-09,18:14:28.9,M014,ON,None,None
1280880,2011-04-15,10:02:18.7,M019,ON,None,None
1315702,2011-04-18,22:51:00.6,M009,OFF,None,None
1433288,2011-05-04,10:39:12,M008,OFF,None,None
1471837,2011-05-10,05:55:16.5,M009,ON,Relax,begin


In [9]:
valid_sensors = [ 'D' + str(i).zfill(3) for i in range(1, 5) ] + \
                [ 'T' + str(i).zfill(3) for i in range(1, 6) ] + \
                [ 'M' + str(i).zfill(3) for i in range(1, 32) ]

# check for invalid sensors
print("Invalid sensors:")
df[~df.sensor.isin(valid_sensors)]

Invalid sensors:


,date,time,sensor,reading,activity,note
1476693,2011-05-10,18:42:45.169231,c,OFF,None,None
1523044,2011-05-17,11:40:03.013619,LEAVEHOME,180,None,None
1523584,2011-05-17,14:58:04.907136,LEAVEHOME,300,None,None
1526729,2011-05-17,18:31:56.044148,LEAVEHOME,300,None,None
1530032,2011-05-18,12:22:23.315366,LEAVEHOME,300,None,None
1530061,2011-05-18,14:12:13.738946,ENTERHOME,6592,None,None


In [10]:
def is_valid_reading(sensor, reading):
    if sensor.startswith('D'):
        return reading in ['OPEN', 'CLOSED', 'CLOSE']
    
    elif sensor.startswith('T'):
        try:
            reading = float(reading)
            return True
        except ValueError:
            return False
      
    elif sensor.startswith('M'):
        return reading in ['ON', 'OFF']
    
    return False

print("Invalid readings:")
df[~df.apply(lambda x: is_valid_reading(x['sensor'], x['reading']), axis=1)]

Invalid readings:


,date,time,sensor,reading,activity,note
275238,2010-12-13,09:42:01.868596,M020,ONc,None,None
275270,2010-12-13,09:44:09.502301,M019,ONc,None,None
275303,2010-12-13,09:46:03.153333,M021,OFFc,None,None
275336,2010-12-13,09:50:05.056768,M012,ONc,None,None
275369,2010-12-13,09:57:01.039203,M012,OFFc,None,None
...,...,...,...,...,...,...
1523044,2011-05-17,11:40:03.013619,LEAVEHOME,180,None,None
1523584,2011-05-17,14:58:04.907136,LEAVEHOME,300,None,None
1526729,2011-05-17,18:31:56.044148,LEAVEHOME,300,None,None
1530032,2011-05-18,12:22:23.315366,LEAVEHOME,300,None,None


--- Issue : 
- Motion readings
  - 'O' instead of 'ON'
  - 'OF' instead of 'OFF'
  - 'c' in the string : OFFcc, OFcF, ONc
  - digit in the string : OFF5
  - sensor id within values: ONM026, ONM024
- Temperature readings
  - 'c' in the string : 28.55c, 26cc
  - outlier values (digits didn't fall within the typical range of 0-50) : 245, 285
- Door readings
  - 'c' in the string : OPENc

In [11]:
df.note.value_counts()

note
begin    6477
end      6477
Name: count, dtype: int64

----

## Data Cleaning

In [12]:
def fix_reading(sensor, reading):
    cleaned_reading = reading.replace('c', '')
    
    if sensor.startswith('D'):
        if cleaned_reading in ['OPEN', 'CLOSED']:
            return cleaned_reading
        elif cleaned_reading == 'CLOSE':
            return 'CLOSED'
    
    elif sensor.startswith('T'):
        try:
            if float(cleaned_reading):
                # Check if the reading is a valid float value for temperature
                if '.' not in cleaned_reading and len(cleaned_reading) > 2:
                    return cleaned_reading[:2] + '.' + cleaned_reading[2:]
                
                return cleaned_reading
        except ValueError:
            pass
      
    elif sensor.startswith('M'):
        if cleaned_reading in ['ON', 'OFF']: return cleaned_reading
        elif cleaned_reading == 'O': return 'ON'
        elif cleaned_reading == 'OF': return 'OFF'
        elif cleaned_reading[:2] == 'ON': return 'ON'
        elif cleaned_reading[:3] == 'OFF': return 'OFF'
    
    return 'INVALID-' + reading

In [13]:
# remove invalid senors 
df_cleaned = df[df.sensor.isin(valid_sensors)]

# filter out invalid readings
df_cleaned['reading'] = df_cleaned.apply(lambda x: fix_reading(x['sensor'], x['reading']), axis=1)

# remove duplicates
df_cleaned = df_cleaned.drop_duplicates(keep='first')

/var/folders/n5/mm68ph654jj0kwghx08j70l40000gp/T/ipykernel_71446/1228275349.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['reading'] = df_cleaned.apply(lambda x: fix_reading(x['sensor'], x['reading']), axis=1)


In [14]:
df_cleaned[df_cleaned['reading'].str.startswith('INVALID-')]

,date,time,sensor,reading,activity,note


In [15]:
# replace None values with Np.nan
df_cleaned.replace(pd.NA, np.nan, inplace=True)

In [16]:
df_cleaned.activity.value_counts()/2

activity
Relax               2907.0
Meal_Preparation    1596.0
Leave_Home           427.0
Enter_Home           427.0
Sleeping             398.0
Eating               255.0
Work                 171.0
Bed_to_Toilet        156.0
Wash_Dishes           64.0
Housekeeping          33.0
Respirate              6.0
Name: count, dtype: float64

In [17]:
df_cleaned

,date,time,sensor,reading,activity,note
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin
1,2010-11-04,00:03:57.399391,M003,OFF,NaN,NaN
2,2010-11-04,00:15:08.984841,T002,21.5,NaN,NaN
3,2010-11-04,00:30:19.185547,T003,21,NaN,NaN
4,2010-11-04,00:30:19.385336,T004,21,NaN,NaN
...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.28507,T002,25.5,NaN,NaN
1719554,2011-06-11,23:48:02.888409,T001,23.5,NaN,NaN
1719555,2011-06-11,23:48:02.988798,T002,25,NaN,NaN
1719556,2011-06-11,23:53:06.4292,T002,25.5,NaN,NaN


--------

## Preprocessing

Standardize temperature readings

In [18]:
def standardize_temp_format(reading, sensor):
  if sensor.startswith('T'):
    # conver the temperature reading to 5 decimal places
    temp = float(reading)
    return f"{temp:.2f}"
  
  return reading

# standardize the temp format
df_cleaned['reading'] = df_cleaned.apply(lambda x: standardize_temp_format(x['reading'], x['sensor']), axis=1)

df_cleaned

,date,time,sensor,reading,activity,note
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin
1,2010-11-04,00:03:57.399391,M003,OFF,NaN,NaN
2,2010-11-04,00:15:08.984841,T002,21.50,NaN,NaN
3,2010-11-04,00:30:19.185547,T003,21.00,NaN,NaN
4,2010-11-04,00:30:19.385336,T004,21.00,NaN,NaN
...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.28507,T002,25.50,NaN,NaN
1719554,2011-06-11,23:48:02.888409,T001,23.50,NaN,NaN
1719555,2011-06-11,23:48:02.988798,T002,25.00,NaN,NaN
1719556,2011-06-11,23:53:06.4292,T002,25.50,NaN,NaN


Standardize Time

In [19]:
def standardize_time(time):
  # convert the time to 24 hour format
  if '.' not in time:
    time = time + '.000000'
  
  else:
    time = time.split('.')
    time = time[0] + '.' + time[1].ljust(6, '0')
  
  return time

# standardize the time format
df_cleaned['time'] = df_cleaned['time'].apply(standardize_time)

df_cleaned

,date,time,sensor,reading,activity,note
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin
1,2010-11-04,00:03:57.399391,M003,OFF,NaN,NaN
2,2010-11-04,00:15:08.984841,T002,21.50,NaN,NaN
3,2010-11-04,00:30:19.185547,T003,21.00,NaN,NaN
4,2010-11-04,00:30:19.385336,T004,21.00,NaN,NaN
...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.285070,T002,25.50,NaN,NaN
1719554,2011-06-11,23:48:02.888409,T001,23.50,NaN,NaN
1719555,2011-06-11,23:48:02.988798,T002,25.00,NaN,NaN
1719556,2011-06-11,23:53:06.429200,T002,25.50,NaN,NaN


In [20]:
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['date'].astype(str) + ' ' + df_cleaned['time'], format='%Y-%m-%d %H:%M:%S.%f')

df_cleaned

,date,time,sensor,reading,activity,note,timestamp
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin,2010-11-04 00:03:50.209589
1,2010-11-04,00:03:57.399391,M003,OFF,NaN,NaN,2010-11-04 00:03:57.399391
2,2010-11-04,00:15:08.984841,T002,21.50,NaN,NaN,2010-11-04 00:15:08.984841
3,2010-11-04,00:30:19.185547,T003,21.00,NaN,NaN,2010-11-04 00:30:19.185547
4,2010-11-04,00:30:19.385336,T004,21.00,NaN,NaN,2010-11-04 00:30:19.385336
...,...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.285070,T002,25.50,NaN,NaN,2011-06-11 23:42:59.285070
1719554,2011-06-11,23:48:02.888409,T001,23.50,NaN,NaN,2011-06-11 23:48:02.888409
1719555,2011-06-11,23:48:02.988798,T002,25.00,NaN,NaN,2011-06-11 23:48:02.988798
1719556,2011-06-11,23:53:06.429200,T002,25.50,NaN,NaN,2011-06-11 23:53:06.429200


In [21]:
df_preprocess = df_cleaned.copy()
df_preprocess.sort_values(by=['timestamp'], inplace=True)

df_preprocess

,date,time,sensor,reading,activity,note,timestamp
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin,2010-11-04 00:03:50.209589
1,2010-11-04,00:03:57.399391,M003,OFF,NaN,NaN,2010-11-04 00:03:57.399391
2,2010-11-04,00:15:08.984841,T002,21.50,NaN,NaN,2010-11-04 00:15:08.984841
3,2010-11-04,00:30:19.185547,T003,21.00,NaN,NaN,2010-11-04 00:30:19.185547
4,2010-11-04,00:30:19.385336,T004,21.00,NaN,NaN,2010-11-04 00:30:19.385336
...,...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.285070,T002,25.50,NaN,NaN,2011-06-11 23:42:59.285070
1719554,2011-06-11,23:48:02.888409,T001,23.50,NaN,NaN,2011-06-11 23:48:02.888409
1719555,2011-06-11,23:48:02.988798,T002,25.00,NaN,NaN,2011-06-11 23:48:02.988798
1719556,2011-06-11,23:53:06.429200,T002,25.50,NaN,NaN,2011-06-11 23:53:06.429200


In [22]:
# find how long it takes for each activity to occur
activities = { x : [] for x in all_activities }
activities["no_activity"] = []

activity_timestamp = {
  x : None for x in all_activities
}
activity_timestamp["no_activity"] = None

for index, row in df_preprocess.iterrows():
  if row['activity'] and row['note'] == 'begin':
    # if the no_activity is not None, calculate the duration
    if activity_timestamp['no_activity'] is not None:
      duration = (row['timestamp'] - activity_timestamp['no_activity']).total_seconds()
      activities['no_activity'].append(duration)
      activity_timestamp['no_activity'] = None
    
    activity_timestamp[row['activity']] = row['timestamp']
  
  elif row['activity'] and row['note'] == 'end':
    cur_activity = row['activity']
    end_time = row['timestamp']
    duration = (end_time - activity_timestamp[cur_activity]).total_seconds()
    activities[cur_activity].append(duration)
    
    activity_timestamp[row['activity']] = None
  
  # if all activities are None, set the no_activity to None to find the time where the person is not doing any activity
  elif all([activity_timestamp[x] is None for x in all_activities]):
    activity_timestamp['no_activity'] = row['timestamp']

In [23]:
data = []

for activity, durations in activities.items():
  data.append([activity, sum(durations), len(durations), np.mean(durations), max(durations), min(durations)])

df_acitivies = pd.DataFrame(data, columns=['activity', 'total_duration', 'count', 'mean_duration', 'max_duration', 'min_duration']).sort_values(by='total_duration', ascending=False)
for col in ['total_duration', 'mean_duration', 'max_duration', 'min_duration']:
  df_acitivies[col] = df_acitivies[col].apply(lambda x: pd.Timedelta(seconds=x))
  
  if col != 'total_duration':
    # put the format as hh:mm:ss
    df_acitivies[col] = df_acitivies[col].dt.components.apply(lambda x: f"{int(x.hours):02}:{int(x.minutes):02}:{int(x.seconds):02}", axis=1)
    

df_acitivies.sort_values(by='mean_duration', ascending=False)

,activity,total_duration,count,mean_duration,max_duration,min_duration
8,Sleeping,66 days 17:00:46.987992,398,04:01:21,10:46:23,00:00:33
6,Relax,67 days 12:14:49.019053,2907,00:33:26,04:15:01,00:00:56
3,Housekeeping,0 days 11:10:41.555535,33,00:20:19,01:30:00,00:01:32
10,Work,2 days 00:40:45.545265,171,00:17:04,01:56:56,00:00:54
1,Eating,1 days 19:08:03.797585,255,00:10:08,01:09:28,00:00:15
7,Respirate,0 days 00:51:23.150781,6,00:08:33,00:24:55,00:01:38
5,Meal_Preparation,8 days 16:37:08.445450,1596,00:07:50,05:26:12,00:00:07
9,Wash_Dishes,0 days 07:43:45.120102,64,00:07:14,00:36:39,00:00:33
0,Bed_to_Toilet,0 days 07:05:47.941873,156,00:02:43,00:08:42,00:00:21
11,no_activity,2 days 05:22:41.747047,6298,00:00:30,01:14:39,00:00:00


In [24]:
# sample time by 60 seconds
df_preprocess['sampled_time'] = df_preprocess['timestamp'].dt.floor('5S')

# check if any activity overlaps
values = df_preprocess.groupby('sampled_time').activity.nunique().reset_index(name='activity_count').sort_values(by='activity_count', ascending=False)
# values.activity_count.value_counts()

values.activity_count.value_counts()

/var/folders/n5/mm68ph654jj0kwghx08j70l40000gp/T/ipykernel_71446/58331170.py:2: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_preprocess['sampled_time'] = df_preprocess['timestamp'].dt.floor('5S')


activity_count
0    680918
1     12538
2       136
Name: count, dtype: int64

In [25]:
# Check if the sensors are duplicated
df_preprocess.groupby('sampled_time')['sensor'].apply(lambda x: len(x) - len(set(x))).value_counts()

sensor
0    392372
1    204750
2     72702
3     18495
4      4194
5       844
6       190
7        33
8         9
9         3
Name: count, dtype: int64

In [26]:
# for i in range(values[values.activity_count == 2].shape[0]):
#   display(df_preprocess[df_preprocess['sampled_time'] == values.iloc[i]['sampled_time']])

In [27]:
# Step 1: Identify active periods based on "begin" and "end"
activity_label = []
current_activity = np.nan

for idx, row in df_preprocess.iterrows():
    if row['note'] == 'begin' and pd.notnull(row['activity']):
        current_activity = row['activity']
    elif row['note'] == 'end' and pd.notnull(row['activity']):
        current_activity = np.nan
    
    activity_label.append(current_activity)

# Add activity column
df_preprocess['activity_label'] = activity_label

df_preprocess

,date,time,sensor,reading,activity,note,timestamp,sampled_time,activity_label
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin,2010-11-04 00:03:50.209589,2010-11-04 00:03:50,Sleeping
1,2010-11-04,00:03:57.399391,M003,OFF,NaN,NaN,2010-11-04 00:03:57.399391,2010-11-04 00:03:55,Sleeping
2,2010-11-04,00:15:08.984841,T002,21.50,NaN,NaN,2010-11-04 00:15:08.984841,2010-11-04 00:15:05,Sleeping
3,2010-11-04,00:30:19.185547,T003,21.00,NaN,NaN,2010-11-04 00:30:19.185547,2010-11-04 00:30:15,Sleeping
4,2010-11-04,00:30:19.385336,T004,21.00,NaN,NaN,2010-11-04 00:30:19.385336,2010-11-04 00:30:15,Sleeping
...,...,...,...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.285070,T002,25.50,NaN,NaN,2011-06-11 23:42:59.285070,2011-06-11 23:42:55,Sleeping
1719554,2011-06-11,23:48:02.888409,T001,23.50,NaN,NaN,2011-06-11 23:48:02.888409,2011-06-11 23:48:00,Sleeping
1719555,2011-06-11,23:48:02.988798,T002,25.00,NaN,NaN,2011-06-11 23:48:02.988798,2011-06-11 23:48:00,Sleeping
1719556,2011-06-11,23:53:06.429200,T002,25.50,NaN,NaN,2011-06-11 23:53:06.429200,2011-06-11 23:53:05,Sleeping


In [49]:
df_pivot = df_preprocess.groupby('sampled_time')['activity_label'].last().reset_index().set_index('sampled_time').replace(np.nan, 'no_activity')

df_pivot

,activity_label
sampled_time,
2010-11-04 00:03:50,Sleeping
2010-11-04 00:03:55,Sleeping
2010-11-04 00:15:05,Sleeping
2010-11-04 00:30:15,Sleeping
2010-11-04 00:35:20,Sleeping
...,...
2011-06-11 23:37:55,Sleeping
2011-06-11 23:42:55,Sleeping
2011-06-11 23:48:00,Sleeping


In [50]:
# Count the number of readings for each sensor
for sensor_type in ['D', 'M', 'T']:
    df_subset = df_preprocess[df_preprocess['sensor'].str.startswith(sensor_type)]

    if sensor_type == 'T':
        # Create bins for temperature readings
        # Ensure the 'reading' column is of type float
        df_subset['reading'] = pd.to_numeric(df_subset['reading'], errors='coerce')

        min_val = df_subset['reading'].min()
        max_val = df_subset['reading'].max()
        bins = pd.cut(df_subset['reading'], bins=10, include_lowest=True)
        
        # make sure bins are within 2 decimal places
        bins = bins.apply(lambda x: f"{x.left:.2f} - {x.right:.2f}" if pd.notnull(x) else x)

        # Replace the reading column with bin labels
        df_subset = df_subset.copy()
        df_subset['reading_bin'] = bins.astype(str)  # use str for column names later

        # Group by bin
        df_temp = (
            df_subset
            .groupby(['sampled_time', 'sensor', 'reading_bin'])
            .size()
            .unstack(fill_value=0)
            .reset_index()
        )

        # Melt and create combined sensor-reading column
        df_temp = df_temp.melt(id_vars=['sampled_time', 'sensor'], var_name='reading', value_name='count')
        df_temp['sensor_reading'] = df_temp['sensor'] + '-' + df_temp['reading']
    else:
        # Group and count ON/OFF readings
        df_temp = (
            df_subset
            .groupby(['sampled_time', 'sensor'])['reading']
            .value_counts()
            .unstack(fill_value=0)
            .reset_index()
        )

        # Melt and create combined sensor-reading column
        df_temp = df_temp.melt(id_vars=['sampled_time', 'sensor'], var_name='reading', value_name='count')
        df_temp['sensor_reading'] = df_temp['sensor'] + '-' + df_temp['reading']

    # Pivot to wide format and merge with main df_pivot
    pivoted = df_temp.pivot(index='sampled_time', columns='sensor_reading', values='count').fillna(0).astype(int)
    df_pivot = df_pivot.merge(pivoted, on='sampled_time', how='left')

df_pivot

/var/folders/n5/mm68ph654jj0kwghx08j70l40000gp/T/ipykernel_71446/2608452540.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['reading'] = pd.to_numeric(df_subset['reading'], errors='coerce')


,activity_label,D001-CLOSED,D001-OPEN,D002-CLOSED,D002-OPEN,D004-CLOSED,D004-OPEN,M001-OFF,M001-ON,M002-OFF,...,T005-15.97 - 18.70,T005-18.70 - 21.40,T005-21.40 - 24.10,T005-24.10 - 26.80,T005-26.80 - 29.50,T005-29.50 - 32.20,T005-32.20 - 34.90,T005-34.90 - 37.60,T005-37.60 - 40.30,T005-40.30 - 43.00
sampled_time,,,,,,,,,,,,,,,,,,,,,
2010-11-04 00:03:50,Sleeping,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-04 00:03:55,Sleeping,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-04 00:15:05,Sleeping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-11-04 00:30:15,Sleeping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-11-04 00:35:20,Sleeping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-06-11 23:37:55,Sleeping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-06-11 23:42:55,Sleeping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-06-11 23:48:00,Sleeping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# put 0 for the missing values
df_pivot.fillna(0, inplace=True)
df_pivot.reset_index(drop=False, inplace=True)

df_pivot

,sampled_time,activity_label,D001-CLOSED,D001-OPEN,D002-CLOSED,D002-OPEN,D004-CLOSED,D004-OPEN,M001-OFF,M001-ON,...,T005-15.97 - 18.70,T005-18.70 - 21.40,T005-21.40 - 24.10,T005-24.10 - 26.80,T005-26.80 - 29.50,T005-29.50 - 32.20,T005-32.20 - 34.90,T005-34.90 - 37.60,T005-37.60 - 40.30,T005-40.30 - 43.00
0,2010-11-04 00:03:50,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-11-04 00:03:55,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-11-04 00:15:05,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-11-04 00:30:15,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-11-04 00:35:20,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693587,2011-06-11 23:37:55,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
693588,2011-06-11 23:42:55,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
693589,2011-06-11 23:48:00,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
693590,2011-06-11 23:53:05,Sleeping,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
df_pivot[[x for x in df_pivot.columns if x.startswith('T')]].sum(axis=1).value_counts()

0.0    648999
1.0     15573
2.0      8306
5.0      7824
4.0      6484
3.0      6406
Name: count, dtype: int64

In [57]:
activity_encoder = {
  x : i for i, x in enumerate(df_pivot['activity_label'].unique())
}

activity_decoder = {
  str(x) : i for i, x in activity_encoder.items()
}

details = {
  'activity_encoder' : activity_encoder,
  'activity_decoder' : activity_decoder,
  'columns' : df_pivot.columns.tolist(),
}


details

{'activity_encoder': {'Sleeping': 0,
  'no_activity': 1,
  'Bed_to_Toilet': 2,
  'Meal_Preparation': 3,
  'Relax': 4,
  'Housekeeping': 5,
  'Eating': 6,
  'Wash_Dishes': 7,
  'Leave_Home': 8,
  'Enter_Home': 9,
  'Work': 10,
  'Respirate': 11},
 'activity_decoder': {'0': 'Sleeping',
  '1': 'no_activity',
  '2': 'Bed_to_Toilet',
  '3': 'Meal_Preparation',
  '4': 'Relax',
  '5': 'Housekeeping',
  '6': 'Eating',
  '7': 'Wash_Dishes',
  '8': 'Leave_Home',
  '9': 'Enter_Home',
  '10': 'Work',
  '11': 'Respirate'},
 'columns': ['sampled_time',
  'activity_label',
  'D001-CLOSED',
  'D001-OPEN',
  'D002-CLOSED',
  'D002-OPEN',
  'D004-CLOSED',
  'D004-OPEN',
  'M001-OFF',
  'M001-ON',
  'M002-OFF',
  'M002-ON',
  'M003-OFF',
  'M003-ON',
  'M004-OFF',
  'M004-ON',
  'M005-OFF',
  'M005-ON',
  'M006-OFF',
  'M006-ON',
  'M007-OFF',
  'M007-ON',
  'M008-OFF',
  'M008-ON',
  'M009-OFF',
  'M009-ON',
  'M010-OFF',
  'M010-ON',
  'M011-OFF',
  'M011-ON',
  'M012-OFF',
  'M012-ON',
  'M013-OFF',
 

In [58]:
import json 

with open('dataset_summary.json', 'w') as file:
  json.dump(details, file, indent=4)

In [59]:
df_output = df_pivot.copy()
df_output['activity_label'] = df_output['activity_label'].map(activity_encoder)

# keep only sampled_time 
df_output[['sampled_time', 'activity_label']].to_csv('time_series.csv', index=False)

In [60]:
columns = df_output.columns.tolist()
df_output = df_output[columns[1:]].to_csv('sensor_readings.csv', index=False)

-------